In [ ]:
import pandas as pd
import numpy as np
import os
import glob

import sys

%load_ext autoreload
%autoreload 2

In [ ]:
PATH_DAILY_KPI_AD_RESULT_DATA = os.path.join("/mnt/c", "Dataset/energy/processed/daily_kpi_ad_result")

df_daily_kpi_anomaly_detection = pd.read_csv(os.path.join(PATH_DAILY_KPI_AD_RESULT_DATA,'daily_kpi_ad_result.csv'))



In [ ]:
df_daily_kpi_anomaly_detection

In [ ]:
PATH_KPI_DATA = os.path.join("/mnt/c", "Dataset/energy/processed/kpi")

df_kpi = pd.read_parquet(os.path.join(PATH_KPI_DATA,'energy_hourly_kpi.parquet'))

In [ ]:
df_kpi.head()

In [ ]:
df_kpi['Date'] = df_kpi['DateTime'].dt.date.astype('str')

In [ ]:
df_kpi.head()

In [ ]:
df_kpi_with_daily_ad_result = df_daily_kpi_anomaly_detection.merge(df_kpi[['AreaName', 'Date', 'DateTime', 'fossil_energy_ratio']], on = ['AreaName', 'Date'], how='right')

In [ ]:
PATH_METRIC_DATA = os.path.join("/mnt/c", "Dataset/energy/processed/aggregated_generation")

df_metric = pd.read_parquet(os.path.join(PATH_METRIC_DATA, 'energy_hourly_generation_per_country.parquet'))

In [ ]:
df_metric.head()

In [ ]:
df_metric['Date'] = df_metric['DateTime'].dt.date.astype('str')

In [ ]:
df_metric_with_daily_kpi_ad_result = df_daily_kpi_anomaly_detection.merge(df_metric, on = ['AreaName', 'Date'], how='right')

In [ ]:
df_metric_with_daily_kpi_ad_result['Daily_Anomaly_fossil_energy_ratio'] = df_metric_with_daily_kpi_ad_result['Daily_Anomaly_fossil_energy_ratio'].fillna('Normal')

In [ ]:
pd.to_datetime(df_metric_with_daily_kpi_ad_result['Date']).dt.month

In [ ]:
df_metric_with_daily_kpi_ad_result = df_metric_with_daily_kpi_ad_result[pd.to_datetime(df_metric_with_daily_kpi_ad_result['Date']).dt.month == 1]

In [ ]:
df_metric_with_daily_kpi_ad_result.head()

In [ ]:
df_metric.columns

In [ ]:
list_metric = [
    'generation.biomass',
    'generation.fossil.brown_coal_lignite',
    'generation.fossil.coal_derived_gas', 'generation.fossil.gas',
    'generation.fossil.hard_coal', 'generation.fossil.oil',
    'generation.fossil.oil_shale', 'generation.fossil.peat',
    'generation.geothermal', 'generation.hydro.pumped_storage',
    'generation.hydro.run_of_river_and_poundage',
    'generation.hydro.water_reservoir', 'generation.marine',
    'generation.nuclear', 'generation.other', 'generation.other.renewable',
    'generation.solar', 'generation.waste', 'generation.wind.offshore',
    'generation.wind.onshore'
    ]

In [ ]:
df_metric_with_daily_kpi_ad_result[list_metric].fillna(0)

In [ ]:
list_metric

In [ ]:
list_metric_fossil = [
    'generation.fossil.brown_coal_lignite',
    'generation.fossil.coal_derived_gas',
    'generation.fossil.gas',
    'generation.fossil.hard_coal',
    'generation.fossil.oil',
    'generation.fossil.oil_shale',
    'generation.fossil.peat'
]

In [ ]:
len(df_metric_with_daily_kpi_ad_result)

In [ ]:
df_metric_with_daily_kpi_ad_result

### t-SNE

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px


features = df_metric_with_daily_kpi_ad_result[list_metric].fillna(0)
labels = df_metric_with_daily_kpi_ad_result['AreaName'].fillna('Normal')


tsne_2d = TSNE(n_components=2, random_state=0)
project_tsne_2d = tsne_2d.fit_transform(features)



In [ ]:

df_plot = df_metric_with_daily_kpi_ad_result.copy()
# df_plot = pd.concat([df_plot, pd.DataFrame(proj_umap_2d)], axis=1)
df_plot[0] = project_tsne_2d[:, 0]
df_plot[1] = project_tsne_2d[:, 1]


fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
    color='AreaName',
    labels={'color': 'AreaName'}
    # color=labels,
    # color='AreaName',
    # labels={'color': 'AreaName'}
)

fig.show()




In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = project_tsne_2d[:, 0]
df_plot[1] = project_tsne_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
    color='Daily_Anomaly_fossil_energy_ratio',
    labels={'color': 'Daily_Anomaly_fossil_energy_ratio'}
)

fig.show()

In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = project_tsne_2d[:, 0]
df_plot[1] = project_tsne_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
)

fig.show()

## UMAP

In [ ]:
df_metric_with_daily_kpi_ad_result.head()

In [ ]:
len(df_metric_with_daily_kpi_ad_result)

In [ ]:
df_plot.tail()

In [ ]:
df_metric_with_daily_kpi_ad_result.head()

In [ ]:
len(df_metric_with_daily_kpi_ad_result)

In [ ]:
len(pd.DataFrame(proj_umap_2d))

In [ ]:
proj_umap_2d[:, 0]

In [ ]:
from umap import UMAP
import plotly.express as px

features = df_metric_with_daily_kpi_ad_result[list_metric].fillna(0)
labels = df_metric_with_daily_kpi_ad_result['AreaName'].fillna('Normal')
# hover_data=df_metric_with_daily_kpi_ad_result[['AreaName', 'DateTime']]

umap_2d = UMAP(n_components=2,
                # init='random', 
                random_state=0)

proj_umap_2d = umap_2d.fit_transform(features)


In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = proj_umap_2d[:, 0]
df_plot[1] = proj_umap_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
    color='AreaName',
    labels={'color': 'AreaName'}

)

fig.show()

In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = proj_umap_2d[:, 0]
df_plot[1] = proj_umap_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
    color='Daily_Anomaly_fossil_energy_ratio',
    labels={'color': 'Daily_Anomaly_fossil_energy_ratio'}

)

fig.show()

In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = proj_umap_2d[:, 0]
df_plot[1] = proj_umap_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric
)

fig.show()

In [ ]:
from umap import UMAP
import plotly.express as px

features = df_metric_with_daily_kpi_ad_result[list_metric].fillna(0)
labels = df_metric_with_daily_kpi_ad_result['AreaName'].fillna('Normal')
# hover_data=df_metric_with_daily_kpi_ad_result[['AreaName', 'DateTime']]

umap_2d = UMAP(n_components=2,
                # init='random', 
                random_state=0)

# proj_umap_2d = umap_2d.fit_transform(features)
umap_2d.fit(features)
proj_umap_2d = umap_2d.transform(features)

In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = proj_umap_2d[:, 0]
df_plot[1] = proj_umap_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
    color='AreaName',
    labels={'color': 'AreaName'}

)

fig.show()

### densMAP

In [ ]:
from umap import UMAP
import plotly.express as px

features = df_metric_with_daily_kpi_ad_result[list_metric].fillna(0)
labels = df_metric_with_daily_kpi_ad_result['AreaName'].fillna('Normal')
# hover_data=df_metric_with_daily_kpi_ad_result[['AreaName', 'DateTime']]

umap_2d = UMAP(n_components=2,
                densmap=True,
                # init='random', 
                random_state=0)

# proj_umap_2d = umap_2d.fit_transform(features)
umap_2d.fit(features)
proj_umap_2d = umap_2d.transform(features)

In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = proj_umap_2d[:, 0]
df_plot[1] = proj_umap_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
    color='AreaName',
    labels={'color': 'AreaName'}

)

fig.show()

In [ ]:
df_plot = df_metric_with_daily_kpi_ad_result.copy()
df_plot[0] = proj_umap_2d[:, 0]
df_plot[1] = proj_umap_2d[:, 1]

fig = px.scatter(
    df_plot, x=0, y=1,
    hover_data=['AreaName', 'Date', 'Daily_Anomaly_fossil_energy_ratio', 'DateTime']+list_metric,
    color='Daily_Anomaly_fossil_energy_ratio',
    labels={'color': 'Daily_Anomaly_fossil_energy_ratio'}

)

fig.show()
